In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import time
from matplotlib.ticker import LinearLocator
import os
import csv

def simulate(numeroPrueba, isCorreolis, withViscosity = False, epsilon = 1, k=1):

    anchox = 100
    anchoy = 20
    dx = 0.2
    dy = 0.2

    numberOfParticlesX = int(anchox/dx)
    numberOfParticlesY = int(anchoy/dy)

    print(numberOfParticlesX)
    print(numberOfParticlesY)

    g = 9.8 

    D = 0

    x_s = (numberOfParticlesX/2)*dx
    y_s = (numberOfParticlesY/2)*dy

    viscocity = 1   

    current_moment = time.time()

    x = np.arange(-x_s,x_s,dx)

    y = np.arange(-y_s,y_s,dy)

    X,Y = np.meshgrid(x,y)
    X = np.transpose(X)
    Y = np.transpose(Y)


    # np.cos(X**2 + Y**2) +1 #
    #H = np.exp(-((X)**2+(Y)**2)/(numberOfParticles))+1 
    b = getBottom(numeroPrueba,numberOfParticlesX,numberOfParticlesY,X,Y)
    H = D+b
    #H = 0.5*np.sin((X)/(3*np.pi))+1
    # np.exp(-((X)**2+(Y)**2)/(numberOfParticles))
    renderBottom(numeroPrueba,H,X,Y, epsilon, k)
    valoresIniciales = np.exp(-((X)**2+(Y)**2)/(epsilon))*np.cos(k*X) + 50 + D
    #valoresIniciales = 0.5*np.sin((X)/(3*np.pi))+3
    #np.exp(-(X**2+Y**2)/numberOfParticles)+1.5

    eta_n = np.copy(valoresIniciales)
    u_n = np.zeros((numberOfParticlesX,numberOfParticlesY))
    v_n = np.zeros((numberOfParticlesX,numberOfParticlesY))

    u_n[:,:] = 0
    v_n[:,:] = 0

    eta_n_1 = np.zeros((numberOfParticlesX,numberOfParticlesY))
    u_n_1 = np.zeros((numberOfParticlesX,numberOfParticlesY))
    v_n_1 = np.zeros((numberOfParticlesX,numberOfParticlesY))

    dt = np.amin(0.01*((dx)/np.sqrt(g*np.abs(H))))

    duration = 10

    fps = 1/30

    FRAMES = duration * (1/fps)

    current_frame = 0

    current_time = 0

    COREOLIS = 2*float(2*np.pi/(24))*np.sin(np.deg2rad(90))
    
    slide = 25

    data = [np.concatenate(([-100], x))]
    
    while(current_frame < FRAMES):
        current_time+= dt

        #### Calculos de u

        u_temp = np.zeros((numberOfParticlesX,numberOfParticlesY))

        if isCorreolis == True:
            u_temp -= v_n * COREOLIS

        if withViscosity == True:
            u_temp[1:-1,:] -= viscocity * ((u_n[:-2,:] - 2*u_n[1:-1,:] + u_n[2:,:])/dx**2)
            u_temp[:,1:-1] -= viscocity * ((u_n[:,:-2] - 2*u_n[:,1:-1] + u_n[:,2:])/dy**2)

        u_temp[:-1,:] += g*((eta_n[1:,:] - eta_n[:-1,:])/(dx))
        #u_temp[:-1,:] += u_n[1:,:] * ((u_n[1:,:] - u_n[:-1,:])/(dx))
        #u_temp[:,:-1] += v_n[:,1:] * ((u_n[:,1:] - u_n[:,:-1])/(dy))
        
        u_n_1 = u_n + dt*u_temp
        u_n_1[-1,:] = 0
        
        #### Calculos de v

        v_temp = np.zeros((numberOfParticlesX,numberOfParticlesY))

        if isCorreolis == True:
            v_temp += u_n * COREOLIS

        if withViscosity == True:
            v_temp[1:-1,:] -= viscocity * ((v_n[:-2,:] - 2*v_n[1:-1,:] + v_n[2:,:])/dx**2)
            v_temp[:,1:-1] -= viscocity * ((v_n[:,:-2] - 2*v_n[:,1:-1] + v_n[:,2:])/dy**2)

        v_temp[:,:-1] += g*((eta_n[:,1:] - eta_n[:,:-1])/(dy))
        #v_temp[:-1,:] += u_n[1:,:] * ((v_n[1:,:] - v_n[:-1,:])/(dx))
        #v_temp[:,:-1] += v_n[:,1:] * ((v_n[:,1:] - v_n[:,:-1])/(dy))
        
        v_n_1 = v_n + dt*v_temp
        v_n_1[:,-1] = 0

        eta_temp = np.zeros((numberOfParticlesX,numberOfParticlesY))

        eta_H = eta_n[:,:] + H[:,:]

        #eta_temp[:-1,:] += u_n[:-1,:]*(eta_n[1:,:] - eta_n[:-1,:])/dx
        #eta_temp[:-1,:] += eta_n[:-1,:]*(u_n[1:,:] - u_n[:-1,:])/dx
        #eta_temp[:-1,:] += u_n[:-1,:]*(H[1:,:] - H[:-1,:])/dx
        #eta_temp[:-1,:] += H[:-1,:]*(u_n[1:,:] - u_n[:-1,:])/dx

        #eta_temp[:,:-1] += v_n[:,:-1]*(eta_n[:,1:] - eta_n[:,:-1])/dy
        #eta_temp[:,:-1] += eta_n[:,:-1]*(v_n[:,1:] - v_n[:,:-1])/dy
        #eta_temp[:,:-1] += v_n[:,:-1]*(H[:,1:] - H[:,:-1])/dy
        #eta_temp[:,:-1] += H[:,:-1]*(v_n[:,1:] - v_n[:,:-1])/dy

        eta_temp[1:,:] += (u_n_1[1:,:] - u_n_1[:-1,:])*(eta_H[1:,:])/dx
        eta_temp[0,:] += (u_n_1[0,:])*(eta_H[0,:])/dx

        eta_temp[1:,:] += (eta_n[1:,:] - eta_n[:-1,:] + H[1:,:] - H[:-1,:])*(u_n_1[1:,:])/dx
        eta_temp[0,:] += (eta_n[0,:] + H[0,:] )*(u_n_1[0,:])/dx

        eta_temp[:,1:] += (v_n_1[:,1:] - v_n_1[:,:-1])*(eta_H[:,1:])/dy
        eta_temp[:,0] += (v_n_1[:,0])*(eta_H[:,0])/dy

        eta_temp[:,1:] += (eta_n[:,1:] - eta_n[:,:-1] + H[:,1:] - H[:,:-1])*(v_n_1[:,1:])/dy
        eta_temp[:,0] += (eta_n[:,0]+ H[:,0] )*(v_n_1[:,0])/dy

        eta_n_1 = eta_n + dt*eta_temp

        eta_n = np.copy(eta_n_1)
        u_n = np.copy(u_n_1)
        v_n = np.copy(v_n_1)

        haveCoreolis = ""
        if isCorreolis == True:
            haveCoreolis = "coreolis_"
        viscocityText = ""
        if withViscosity == True:
            viscocityText = "viscocity_"
        if(current_time > fps):
            current_time = 0
            current_frame+=1
            time_ = time.time()
            #data.append(np.concatenate(([current_frame],eta_n[slide,:])))
            #fig = plt.figure(num=1, clear=True,dpi=100, figsize=(10,10))
            #ax = fig.add_subplot(projection='3d')
            #ax.set_zlim(0,30)

            print(str(numeroPrueba) + " frame " + str(current_frame) + " " + haveCoreolis + " {000:.1f}".format((current_frame/FRAMES)*100) + "%" + " ETA " +  "{0000:.0f}".format(((FRAMES - current_frame) * (time_ - current_moment))) + " s")

            #ax.plot_surface(X, Y, eta_n , cmap=cm.coolwarm, linewidth=0, antialiased=True)
            #ax.plot_surface(X, Y, H, cmap=cm.coolwarm_r, linewidth=0, antialiased=True)
            
            #plt.savefig( str(numeroPrueba) + '/frame_' + str(haveCoreolis) + str(current_frame) + '.jpg')

            current_moment = time_

            render(X,Y,eta_n,isCorreolis,withViscosity ,numeroPrueba,current_frame,H, epsilon, k)
            #renderVectorField(X,Y,u_n,v_n,current_frame,numeroPrueba,isCorreolis)

    with open(str(numeroPrueba) +'/' + str(epsilon) + '_' + str(k) +'/01-ResultadosAmplitud' + haveCoreolis+ viscocityText +'.csv', 'w', newline='') as f:
        writer = csv.writer(f)
            # write the data
        writer.writerows(data)

    print('終わり')

def render(X,Y,eta_n, isCoriolis,withViscocity ,numeroP, current_frame,B, epsilon, k):
    fig, ax = plt.subplots(num=1,clear=True,subplot_kw={"projection": "3d"})
    ax.set_zlim(0,100)
    ax.set_box_aspect((5,1,1))
    fig.set_size_inches(8,8)
    fig.set_dpi(100)
    ax.plot_surface(X, Y, eta_n , cmap=cm.coolwarm, linewidth=0, antialiased=True)
    ax.plot_surface(X, Y, B, cmap=cm.coolwarm_r, linewidth=0, antialiased=True)
    haveCoreolis = ""
    viscocityText = ""
    if isCoriolis == True:
        haveCoreolis = "coreolis_"
    if withViscocity == True:
        viscocityText = "viscocity_"
    plt.savefig( str(numeroP) + '/' + str(epsilon) + '_' + str(k) + '/frame_' + str(haveCoreolis) + str(viscocityText) + str(current_frame) + '.jpg')
    fig.clear()
    plt.close(fig)
    plt.clf()

def renderVectorField(X,Y,U,V, frame, numeroP, isCoriolis):
    fig, ax = plt.subplots()
    fig.set_size_inches(8,8)
    fig.set_dpi(100)
    ax.quiver(X, Y, U, V)
    haveCoreolis = ""
    if isCoriolis == True:
        haveCoreolis = "coreolis_"
    plt.savefig( str(numeroP) + '/vectorfield_frame_' + str(haveCoreolis) + str(frame) + '.jpg')
    fig.clear()
    plt.close()
    

def getBottom(prueba,numberOfParticlesX,numberOfParticlesY,X,Y):
    H = np.ones((numberOfParticlesX,numberOfParticlesY))
    if prueba == "Prueba_2":
        H[45:55,:] = 2
    elif prueba == "Prueba_3":
        H = 20 * np.exp(-((X)**2)/(numberOfParticlesX))+1 
    elif prueba == "Prueba_4":
        H = np.exp(-((Y)**2)/(numberOfParticlesY))+1 
    elif prueba == "Prueba_5":
        H = np.exp(-((X)**2 + Y**2)/(numberOfParticlesX))+0.5 
    elif prueba == "Prueba_6":
        H = 0.5*np.sin((X)/(2*np.pi))+1
    elif prueba == "Prueba_7":
        H = 0.5*np.sin((Y)/(3*np.pi))+1
    elif prueba == "Prueba_8":
        H = 0.5*np.sin((X+Y)/(3*np.pi))+1
    elif prueba == "Prueba_9":
        H = 0.5*np.sin((X+Y)**2/(3*np.pi))+1
    elif prueba == "Prueba_10":
        H = np.exp(-((X)**2 + Y**2)/(numberOfParticlesX)) * 0.5*np.sin((X+Y)**2/(3*np.pi))+1
    
    return H

    
def renderBottom(prueba,bottom, X, Y, epsilon, k):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

    ax.set_zlim(0,100)
    ax.set_box_aspect((5,1,1))
    fig.set_size_inches(8,8)
    fig.set_dpi(100)
    ax.plot_surface(X, Y, bottom , cmap=cm.coolwarm, linewidth=0, antialiased=False)

    plt.savefig( str(prueba)+ '/' + str(epsilon) + '_' + str(k)  + '/fondo.jpg')


simulate("Prueba_1",False,False,0.5,0.5)
simulate("Prueba_1",False,False,1,1)
simulate("Prueba_1",False,False,10,10)
#simulate("Prueba_2",False,True)
#simulate("Prueba_2",True,False)
#simulate("Prueba_2",True,True)
#simulate("Prueba_5",False,False)
#simulate("Prueba_5",False,True)
#simulate("Prueba_5",True,False)
#simulate("Prueba_5",True,True) 
#simulate("Prueba_1",True)
#simulate("Prueba_2",False)
#simulate("Prueba_2",True)
#simulate("Prueba_3",False)
#simulate("Prueba_3",True)
#simulate("Prueba_4",False)
#simulate("Prueba_4",True)
#simulate("Prueba_5",False)
#simulate("Prueba_5",True)
#simulate("Prueba_6",False)
#simulate("Prueba_6",True)
#simulate("Prueba_7",False)
#simulate("Prueba_7",True)
#simulate("Prueba_8",False)
#simulate("Prueba_8",True)
#simulate("Prueba_9",False)
#simulate("Prueba_9",True)
#simulate("Prueba_10",False)
#simulate("Prueba_10",True)

500
100
Prueba_1 frame 1  0.3% ETA 162 s
Prueba_1 frame 2  0.7% ETA 238 s
Prueba_1 frame 3  1.0% ETA 207 s
Prueba_1 frame 4  1.3% ETA 194 s
Prueba_1 frame 5  1.7% ETA 195 s
Prueba_1 frame 6  2.0% ETA 191 s
Prueba_1 frame 7  2.3% ETA 224 s
Prueba_1 frame 8  2.7% ETA 181 s
Prueba_1 frame 9  3.0% ETA 179 s
Prueba_1 frame 10  3.3% ETA 179 s
Prueba_1 frame 11  3.7% ETA 181 s
Prueba_1 frame 12  4.0% ETA 217 s
Prueba_1 frame 13  4.3% ETA 177 s
Prueba_1 frame 14  4.7% ETA 176 s
Prueba_1 frame 15  5.0% ETA 175 s
Prueba_1 frame 16  5.3% ETA 175 s
Prueba_1 frame 17  5.7% ETA 212 s
Prueba_1 frame 18  6.0% ETA 173 s
Prueba_1 frame 19  6.3% ETA 174 s
Prueba_1 frame 20  6.7% ETA 174 s
Prueba_1 frame 21  7.0% ETA 171 s
Prueba_1 frame 22  7.3% ETA 220 s
Prueba_1 frame 23  7.7% ETA 173 s
Prueba_1 frame 24  8.0% ETA 173 s
Prueba_1 frame 25  8.3% ETA 171 s
Prueba_1 frame 26  8.7% ETA 169 s
Prueba_1 frame 27  9.0% ETA 216 s
Prueba_1 frame 28  9.3% ETA 169 s
Prueba_1 frame 29  9.7% ETA 172 s
Prueba_1 frame 

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

In [3]:
import cv2

def generarVideoSimulacion(prueba, coreolis,viscosity, tamano, epsilon, k):
    haveCoriolis = ""
    if coreolis:
        haveCoriolis = "coreolis_"
    viscocityText = ""
    if viscosity == True:
        viscocityText = "viscocity_"

    imgSize = cv2.imread(str(prueba) + '/' + str(epsilon) + '_' + str(k) +str("/frame_") + str(haveCoriolis)+str(viscocityText)+str(1)+str(".jpg"))

    heigth, width = imgSize.shape[:2]

    video = cv2.VideoWriter('Videos/' + str(prueba) + str(epsilon) + '_' + str(k) + str(haveCoriolis) + str(viscocityText) + '.wmv',cv2.VideoWriter_fourcc(*'mp4v'),15,(width,heigth))
    
    for i in range(1,tamano):
        img = cv2.imread(str(prueba) + '/' + str(epsilon) + '_' + str(k) + str("/frame_") + str(haveCoriolis)+str(viscocityText)+str(i)+str(".jpg"))
        video.write(img)
        print(str(prueba) + str(haveCoriolis) + " " + "{000:.1f}".format(i/tamano * 100) + "%")

    video.release()

#generarVideoSimulacion("Prueba_1", False, 600)
#generarVideoSimulacion("Prueba_1", True, 600)

#generarVideoSimulacion("Prueba_2", False, 600)
#generarVideoSimulacion("Prueba_2", True, 600)

#generarVideoSimulacion("Prueba_3", False, 600)
#generarVideoSimulacion("Prueba_3", True, 600)

#generarVideoSimulacion("Prueba_4", False, 600)
#generarVideoSimulacion("Prueba_4", True, 600)

#generarVideoSimulacion("Prueba_5", False, 600)
#generarVideoSimulacion("Prueba_5", True, 600)

generarVideoSimulacion("Prueba_1",False,False,300,0.5,0.5)
generarVideoSimulacion("Prueba_2",False,False,300,1,1)
generarVideoSimulacion("Prueba_3",False,False,300,10,10)
#generarVideoSimulacion("Prueba_2",False,True,900)
#generarVideoSimulacion("Prueba_2",True,False,900)
#generarVideoSimulacion("Prueba_2",True,True,900)
#generarVideoSimulacion("Prueba_5",False,False,900)
#generarVideoSimulacion("Prueba_5",False,True,900)
#generarVideoSimulacion("Prueba_5",True,False,900)
#generarVideoSimulacion("Prueba_5",True,True,900) 

#generarVideoSimulacion("Prueba_6", False, 600)
#generarVideoSimulacion("Prueba_6", True, 600)

#generarVideoSimulacion("Prueba_7", False, 600)
#generarVideoSimulacion("Prueba_7", True, 600)

#generarVideoSimulacion("Prueba_8", False, 600)
#generarVideoSimulacion("Prueba_8", True, 600)

#generarVideoSimulacion("Prueba_9", False, 600)
#generarVideoSimulacion("Prueba_9", True, 600)

#generarVideoSimulacion("Prueba_10", False, 600)
#generarVideoSimulacion("Prueba_10", True, 600)


Prueba_3 0.3%
Prueba_3 0.7%
Prueba_3 1.0%
Prueba_3 1.3%
Prueba_3 1.7%
Prueba_3 2.0%
Prueba_3 2.3%
Prueba_3 2.7%
Prueba_3 3.0%
Prueba_3 3.3%
Prueba_3 3.7%
Prueba_3 4.0%
Prueba_3 4.3%
Prueba_3 4.7%
Prueba_3 5.0%
Prueba_3 5.3%
Prueba_3 5.7%
Prueba_3 6.0%
Prueba_3 6.3%
Prueba_3 6.7%
Prueba_3 7.0%
Prueba_3 7.3%
Prueba_3 7.7%
Prueba_3 8.0%
Prueba_3 8.3%
Prueba_3 8.7%
Prueba_3 9.0%
Prueba_3 9.3%
Prueba_3 9.7%
Prueba_3 10.0%
Prueba_3 10.3%
Prueba_3 10.7%
Prueba_3 11.0%
Prueba_3 11.3%
Prueba_3 11.7%
Prueba_3 12.0%
Prueba_3 12.3%
Prueba_3 12.7%
Prueba_3 13.0%
Prueba_3 13.3%
Prueba_3 13.7%
Prueba_3 14.0%
Prueba_3 14.3%
Prueba_3 14.7%
Prueba_3 15.0%
Prueba_3 15.3%
Prueba_3 15.7%
Prueba_3 16.0%
Prueba_3 16.3%
Prueba_3 16.7%
Prueba_3 17.0%
Prueba_3 17.3%
Prueba_3 17.7%
Prueba_3 18.0%
Prueba_3 18.3%
Prueba_3 18.7%
Prueba_3 19.0%
Prueba_3 19.3%
Prueba_3 19.7%
Prueba_3 20.0%
Prueba_3 20.3%
Prueba_3 20.7%
Prueba_3 21.0%
Prueba_3 21.3%
Prueba_3 21.7%
Prueba_3 22.0%
Prueba_3 22.3%
Prueba_3 22.7%
Prueba_3 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def generarImagenesDeAmplitud(numeroPrueba,isCoriolis):
    
    limSup = 21
    limInf = 19

    haveCoreolis = ""
    if isCoriolis == True:
        haveCoreolis = "coreolis_"
    data = pd.read_csv(str(numeroPrueba) +'/01-ResultadosAmplitud' + haveCoreolis+'.csv')

    data_np = data.to_numpy()
    data_header = [float(i) for i in list(data.columns)]
    
    rows = np.shape(data_np)[0]
    columns = np.shape(data_np)[1]

    for column in range(1,columns):
        X = []
        Y = []
        for row in range(0,rows):
            X.append(data_np[row,0])
            Y.append(data_np[row,column])
        plt.figure(num=1, clear=True,dpi=100, figsize=(10,10))
        plt.plot(X,Y)
        plt.ylim(limInf,limSup)
        plt.title("Amplitud en " + str(data_header[column]))
        plt.savefig( str(numeroPrueba) + '/amplitud_' + str(haveCoreolis) + str(data_header[column]) + '.jpg')
        plt.grid()
        plt.clf()

    for row in range(0, rows):
        frame = data_np[row,0]
        X = data_header[1:]
        Y = data_np[row,1:]
        plt.plot(X,Y)
        plt.ylim(limInf,limSup)
        plt.grid()
        plt.savefig( str(numeroPrueba) + '/eta_' + str(haveCoreolis) + str(int(frame)) + '.jpg')
        plt.clf()

    print("listo " + str(numeroPrueba) + " " + str(haveCoreolis))

generarImagenesDeAmplitud("Prueba_1",False)
generarImagenesDeAmplitud("Prueba_1",True)
generarImagenesDeAmplitud("Prueba_2",False)
generarImagenesDeAmplitud("Prueba_2",True)
generarImagenesDeAmplitud("Prueba_3",False)
generarImagenesDeAmplitud("Prueba_3",True)
generarImagenesDeAmplitud("Prueba_4",False)
generarImagenesDeAmplitud("Prueba_4",True)
generarImagenesDeAmplitud("Prueba_5",False)
generarImagenesDeAmplitud("Prueba_5",True)
generarImagenesDeAmplitud("Prueba_6",False)
generarImagenesDeAmplitud("Prueba_6",True)
generarImagenesDeAmplitud("Prueba_7",False)
generarImagenesDeAmplitud("Prueba_7",True)
generarImagenesDeAmplitud("Prueba_8",False)
generarImagenesDeAmplitud("Prueba_8",True)
generarImagenesDeAmplitud("Prueba_9",False)
generarImagenesDeAmplitud("Prueba_9",True)
generarImagenesDeAmplitud("Prueba_10",False)
generarImagenesDeAmplitud("Prueba_10",True)

In [ ]:
import cv2 


def generarVideoSimulacionSlide(prueba, coreolis, tamano):
    haveCoriolis = ""
    if coreolis:
        haveCoriolis = "coreolis_"

    imgSize = cv2.imread(str(prueba)+str("/eta_") + str(haveCoriolis)+str(1)+str(".jpg"))

    heigth, width = imgSize.shape[:2]

    video = cv2.VideoWriter('Videos/' + str(prueba) + str(haveCoriolis) +  'slide.wmv',cv2.VideoWriter_fourcc(*'mp4v'),30,(width,heigth))
    
    for i in range(1,tamano):
        img = cv2.imread(str(prueba)+str("/eta_") + str(haveCoriolis)+str(i)+str(".jpg"))
        video.write(img)
        print(str(prueba) + str(haveCoriolis) + " " + "{000:.1f}".format(i/tamano * 100) + "%")

    video.release()

generarVideoSimulacionSlide("Prueba_1", False, 600)
generarVideoSimulacionSlide("Prueba_1", True, 600)

generarVideoSimulacionSlide("Prueba_2", False, 600)
generarVideoSimulacionSlide("Prueba_2", True, 600)

generarVideoSimulacionSlide("Prueba_3", False, 600)
generarVideoSimulacionSlide("Prueba_3", True, 600)

generarVideoSimulacionSlide("Prueba_4", False, 600)
generarVideoSimulacionSlide("Prueba_4", True, 600)

generarVideoSimulacionSlide("Prueba_5", False, 600)
generarVideoSimulacionSlide("Prueba_5", True, 600)

generarVideoSimulacionSlide("Prueba_6", False, 600)
generarVideoSimulacionSlide("Prueba_6", True, 600)

generarVideoSimulacionSlide("Prueba_7", False, 600)
generarVideoSimulacionSlide("Prueba_7", True, 600)

generarVideoSimulacionSlide("Prueba_8", False, 600)
generarVideoSimulacionSlide("Prueba_8", True, 600)

generarVideoSimulacionSlide("Prueba_9", False, 600)
generarVideoSimulacionSlide("Prueba_9", True, 600)

generarVideoSimulacionSlide("Prueba_10", False, 600)
generarVideoSimulacionSlide("Prueba_10", True, 600)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

dx = 1
dy = 1

numberOfParticles = 10

x_s = (numberOfParticles/2)*dx
y_s = (numberOfParticles/2)*dy

x = np.arange(0,2*x_s,dx)

y = np.arange(0,2*y_s,dy)

X,Y = np.meshgrid(x,y)
X = np.transpose(X)
Y = np.transpose(Y)

valoresIniciales = np.exp(-(((X-5)**2+(Y-5)**2)/(5)))

fig, ax = plt.subplots(num=1,clear=True,subplot_kw={"projection": "3d"})
ax.set_box_aspect((1,1,1))
fig.set_size_inches(8,8)
fig.set_dpi(100)
plt.xlabel("x")
plt.ylabel("y")
ax.plot_surface(X, Y, valoresIniciales , cmap=cm.coolwarm, linewidth=0, antialiased=True)

dxx = np.zeros((numberOfParticles,numberOfParticles))
dxx[1:-1,:] = ((valoresIniciales[:-2,:] - 2*valoresIniciales[1:-1,:] + valoresIniciales[2:,:])/dx**2)

fig, ax = plt.subplots(num=2,clear=True,subplot_kw={"projection": "3d"})
ax.set_box_aspect((1,1,1))
fig.set_size_inches(8,8)
fig.set_dpi(100)
plt.xlabel("x")
plt.ylabel("y")
plt.title("dxx")
ax.plot_surface(X, Y, dxx , cmap=cm.coolwarm, linewidth=0, antialiased=True)

dyy = np.zeros((numberOfParticles,numberOfParticles))
dyy[:,1:-1] = ((valoresIniciales[:,:-2] - 2*valoresIniciales[:,1:-1] + valoresIniciales[:,2:])/dy**2)

fig, ax = plt.subplots(num=3,clear=True,subplot_kw={"projection": "3d"})
ax.set_box_aspect((1,1,1))
fig.set_size_inches(8,8)
fig.set_dpi(100)
plt.title("dyy")
plt.xlabel("x")
plt.ylabel("y")
ax.plot_surface(X, Y, dyy , cmap=cm.coolwarm, linewidth=0, antialiased=True)

